In [11]:
import numpy as np

from os.path import exists
from umbridge import serve_model

import utility_functions
import eeg_model


In [12]:
# Set paths
path_electrodes = [
        "data/electrodes.npz",
        "data/electrodes.npz",
        "data/electrodes.npz"]

path_conductivities = 'data/conductivities.txt'

path_leadfield_matrices = [
        "data/leadfield_matrix_1.npz",
        "data/leadfield_matrix_2.npz",
        "data/leadfield_matrix_3.npz"]

path_meshs = [
        "data/mesh_1.msh",
        "data/mesh_2.msh",
        "data/mesh_3.msh"]

l = len(path_leadfield_matrices)
assert l == len(path_meshs)
assert l == len(path_electrodes)

m = len(path_electrodes)

In [13]:
# Set parameters
center = [127,127]
radii = [92,86,78]
conductivities = [0.00043,0.00001,0.00179]

# Set dipole
position = [150,150]
s_ref = utility_functions.get_dipole(position,center)

# Set noise
relative_noise = 0.05

In [14]:
# Create leadfield matrices if not already existing
for i in range(l):
    if not exists(path_leadfield_matrices[i]):
        utility_functions.save_leadfield_matrix(
            path_electrodes, 
            path_conductivities, 
            path_meshs[i], 
            path_leadfield_matrices[i])

In [16]:
# Generate reference values
b_ref = np.zeros(l)
sigma = np.zeros(l)

b_ref, sigma = utility_functions.calc_disturbed_sensor_values(s_ref, path_electrodes[0], relative_noise)

for i in range(1,l):
    print(i)
    if path_electrodes[i] != path_electrodes[0]:
        b_ref[i], sigma[i] = utility_functions.calc_disturbed_sensor_values(s_ref, path_electrodes[i], relative_noise)
    else:
        b_ref[i], sigma[i] = b_ref[0], sigma[0] 

#################################################################
Simulate disturbed sensor values for a given dipole.
################################################################# 

s_ref = Dipole[position: [150 150] moment: [0.707107 0.707107]] 

Reading 2d Gmsh grid...
version 2.2 Gmsh file detected
file contains 2109 nodes
file contains 4096 elements
number of real vertices = 2109
number of boundary elements = 0
number of elements = 4096
volume_conductor.time_reading_gmsh = 0.117092
tensors = 3
time = 0.0001147
volume_conductor.time_reading_tensors = 0.0002418
volume_conductor.time_reordering_indices = 0.0011163
volume_conductor.time = 0.11845
time_element_neighborhood_map 0.0024427 s
degree = 1
element_type = tetrahedron
=== matrix setup (max) 0.0158621 s
[[-2.6490435973413793, 2.5867621101648157, -2.9609352850322703, -3.1751733828149584, -3.2934281682318645, -3.3167187142287777, -3.2452975554738774, -3.078620487804981, -2.815459494930698, -2.4541860108382556, -1.9932963688572

IndexError: invalid index to scalar variable.

In [ ]:
# Create EEG Model
testmodel = eeg_model.EEGModelFromFile(
    b_ref, 
    sigma, 
    path_leadfield_matrices, 
    path_meshs)

# send via localhost
serve_model(testmodel, 4243) 